<a href="https://colab.research.google.com/github/sojeongii/cvcvcv/blob/main/cv_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 드라이브 마운트

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# 경로 지정


In [2]:
%cd drive/My Drive/test
!pwd
!ls -la

/content/drive/My Drive/test
/content/drive/My Drive/test
total 24
drwx------ 2 root root 4096 May 29 08:34 checkpoints
-rw------- 1 root root 7760 May 29 08:38 cv_test.ipynb
drwx------ 2 root root 4096 May 29 08:29 data
drwx------ 2 root root 4096 May 29 08:24 model
drwx------ 2 root root 4096 May 28 08:23 test_dataset


# 압축 풀기

In [ ]:
import tqdm
import zipfile
import os
import glob

# 압축 풀기
file_name = 'test_dataset'
!unzip -qq '{file_name}'



# Test & Predict

In [3]:
from data.dataset import ColorHintDataset
import torch
import torch.utils.data as data
import cv2
import tqdm
import os
from data.dataset import tensor2im
from model.res_unet.ra_unet import ResAttdUNet
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow

device = "cpu"
if torch.cuda.is_available():
  device = "cuda:0"
  print('device 0 :', torch.cuda.get_device_name(0))
  
def main():
    # Change to your data root directory
    root_path = "./test_dataset"

    # Depend on runtime setting
    use_cuda = True

    test_dataset = ColorHintDataset(root_path, 256, 'test')

    dataloaders = {}
    dataloaders['test'] = torch.utils.data.DataLoader(test_dataset, batch_size = 1, shuffle=False)
    print('test dataset: ', len(test_dataset))


    # state_dict = torch.load(check_point)
    model = Unet().to(device)
    model.load_state_dict(torch.load('./checkpoints/model-epoch-29-losses-0.00804.pth'))

    os.makedirs('./outputs', exist_ok=True)
    os.makedirs('./outputs/test', exist_ok=True)

    model.eval()
    with torch.no_grad():
      for i, data in enumerate(tqdm.tqdm(dataloaders['test'])):
          if use_cuda:
              l = data["l"].to(device)
              hint = data["hint"].to(device)
              file_name = data["file_name"][0]

          hint_image = torch.cat((l, hint), dim=1)
          hint_np = tensor2im(hint_image)
          hint_image = hint_image.float().to(device)

          output = model(hint_image).squeeze(1)
          output = torch.cat((l, output), dim = 1)
          out_hint_np = tensor2im(output)

          hint_bgr = cv2.cvtColor(hint_np, cv2.COLOR_LAB2BGR)
          out_hint_bgr = cv2.cvtColor(out_hint_np, cv2.COLOR_LAB2BGR)

          fname = str(file_name).replace("['", '')
          fname = fname.replace("']", '')

          cv2.imwrite("./outputs/test/"+fname, out_hint_bgr)

if __name__ == '__main__':
    main()

ImportError: ignored